In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import truncnorm

In [2]:
# X = pd.DataFrame(np.linspace(0, 1, num=3).reshape(-1, 1), columns=['Dosage'])
# X
X = pd.DataFrame(np.array([[0, 1, 1, 0]]).T)
X

,0
0,0
1,1
2,1
3,0


In [3]:
target = np.array([0, 1, 1, 0]).reshape(-1, 1)
target

array([[0],
       [1],
       [1],
       [0]])

# 1. Initialize Network

In [4]:
def init_weights(X, n_neurons, random_state=42):
    n_inputs = int(X.shape[1])
    stddev = 2 / np.sqrt(n_inputs + n_neurons)
    b = np.zeros(n_neurons)
    weights = truncnorm.rvs(-1, 1, size=(n_inputs, n_neurons), scale=stddev, random_state=random_state)
    
    return pd.DataFrame(weights), b

In [5]:
weights, biases = init_weights(X, 2)
print(weights)
biases

          0         1
0 -0.249842  1.003948


array([0., 0.])

# 2. Forward Propagate

In [6]:
def softmax(x):
    return np.log(1 + np.exp(x))

In [7]:
def crossenthropy(x):
    return 1. / (1. + np.exp(-x))

In [8]:
X

,0
0,0
1,1
2,1
3,0


In [9]:
weights

,0,1
0,-0.249842,1.003948


In [10]:
np.dot(X, weights)

array([[ 0.        ,  0.        ],
       [-0.24984201,  1.00394781],
       [-0.24984201,  1.00394781],
       [ 0.        ,  0.        ]])

In [11]:
np.dot(X, weights) + biases

array([[ 0.        ,  0.        ],
       [-0.24984201,  1.00394781],
       [-0.24984201,  1.00394781],
       [ 0.        ,  0.        ]])

In [12]:
def neuron_layer(X, weights, b, activation=None):
    Z = np.dot(X, weights) + b
    if activation is not None:
        Z = activation(Z)
        pass
    Z = pd.DataFrame(Z)
    return Z

In [13]:
hidden1 = neuron_layer(X, weights, biases, activation=None)
hidden1

,0,1
0,0.000000,0.000000
1,-0.249842,1.003948
2,-0.249842,1.003948
3,0.000000,0.000000


In [14]:
weights_2, biases_2 = init_weights(hidden1, 1, random_state=43)
display(weights_2)
biases_2

,0
0,-0.825995
1,0.216780


array([0.])

In [15]:
output = neuron_layer(hidden1, weights_2, biases_2, activation=None)
output

,0
0,0.000000
1,0.424004
2,0.424004
3,0.000000


# 3. Back Propagate Error

## Sum of squared residuals

$$\Large SSR = \sum_{i-1}^n{(Observed_i - Predicted_i)^2}$$

In [16]:
target = pd.DataFrame(target)
target

,0
0,0
1,1
2,1
3,0


In [17]:
output

,0
0,0.000000
1,0.424004
2,0.424004
3,0.000000


In [18]:
def ssr(target, predicted):
    return np.sum((target - predicted) ** 2) / len(target)

## Derivative of b3

$$\Large \frac{dSSR}{db_3} = \frac{dSSR}{d Predicted} \cdot \frac{d Predicted}{d b_3}$$


$$\Large \frac{dSSR}{db_3} = \sum_{i=1}^n{-2 \cdot (Observed_i - Predicted_i)} \cdot 1$$

In [19]:
def derivative_of_b3_test(target, output):
    summa = 0
    for i in range(len(target)):
        summa += (-2) * (target.iloc[i, 0] - output.iloc[i, 0])
    return summa

In [20]:
derivative_of_b3_test(target, output)

-2.303983938447467

In [21]:
def derivative_of_b3(target, output):
    return np.sum((-2) * (target - output))[0] / len(target)

In [22]:
derivative_of_b3(target, output)

-0.5759959846118667

## Derivative of w3 and w4

$$\Large \frac{dSSR}{dw_3} = \frac{dSSR}{dPredicted} \cdot \frac{dPredicted}{dw_3}$$

$$\Large \frac{dSSR}{dw_4} = \frac{dSSR}{dPredicted} \cdot \frac{dPredicted}{dw_4}$$

$$\Large \frac{dSSR}{dPredicted} = \sum_{i=1}^n{-2 \cdot (Observed_i - Predicted_i)}$$

$$\Large \frac{dPredicted}{dw_3} = \frac{d}{dw_3}(y_{1,i}w_3 + y_{2,i}w_4 + b_3) = y_{1,i}$$


$$\Large \frac{dPredicted}{dw_4} = \frac{d}{dw_4}(y_{1,i}w_3 + y_{2,i}w_4 + b_3) = y_{2,i}$$

In [23]:
##-- запасной вариант: на выходе один и тот же ответ


def derivative_weights_test(target, output, hidden1):
    derivative = []
    for i in range(hidden1.shape[1]):
        summa = 0
        for j in range(len(output)):
            summa += (-2.) * (target.iloc[j, 0] - output.iloc[j, 0]) * hidden1.iloc[j, i]
        derivative.append(summa)
    return pd.DataFrame(derivative)

gradient_weights = derivative_weights_test(target, output, hidden1)
gradient_weights

,0
0,0.575632
1,-2.313080


In [24]:
def derivative_weights(target, output, hidden1):
    return np.dot(((-2) * (target - output)).T / len(target), hidden1).T

In [25]:
gradient_weights = derivative_weights(target, output, hidden1)
gradient_weights

array([[ 0.143908  ],
       [-0.57826991]])

## Derivative of w1 and w2 and b1,b2

$$\Large \frac{dSSR}{dw_1} = \frac{dSSR}{dPredicted} \cdot \frac{dPredicted}{dy_1} \cdot \frac{dy_1}{dx_1} \cdot \frac{dx_1}{dw_1}$$

$$\Large \frac{dSSR}{dw_2} = \frac{dSSR}{dPredicted} \cdot \frac{dPredicted}{dy_2} \cdot \frac{dy_2}{dx_2} \cdot \frac{dx_2}{dw_2}$$

$$\Large \frac{dSSR}{dPredicted} = \sum_{i=1}^n{-2 \cdot (Observed_i - Predicted_i)}$$

$$\Large \frac{dPredicted}{dy_1} = \frac{d}{dy_1}(y_{1,i}w_3 + y_{2,i}w_4 + b_3) = w_3$$


$$\Large \frac{dy_1}{dx_1} = \frac{d}{dx_1}ln(1 + e^x) = \frac{e^x}{e^x + 1}$$

$$\Large \frac{dx_1}{dw_1} = \frac{d}{dw_1}(Input_i \cdot w_1 + b_1) = Input_i$$

In [229]:
def softmax_derivative(x):
    return np.exp(x) / (1 + np.exp(x))

In [227]:
##-- еще один вариант hidden_1

# def derivative_hidden_2(target, output, weights_2, weights, biases, X):
#     ssr_pred = ((-2.) * (target - output)) / len(target)
    
#     expon = softmax_derivative(X.dot(weights) + biases)
#     weights_deriv = []
#     for i in range(len(weights.T)):
#         weights_deriv.append( np.dot(( ssr_pred * pd.DataFrame(expon[i]).values * weights_2.iloc[i,0] ).T, X)[0] )
    
#     biases_deriv = np.dot(ssr_pred.T, expon) * weights_2.T 
#     return pd.DataFrame(weights_deriv)#, np.array(biases_deriv)

In [228]:
# derivative_hidden_2(target, output, weights_2, weights, biases, X)

,0
0,0.208322
1,-0.091380


In [137]:
def derivative_hidden_1(target, output, weights_2, weights, biases, X):
    ssr_pred = ((-2.) * (target - output)) / len(target)
    
    expon = softmax_derivative(X.dot(weights) + biases)
    weights_deriv = np.dot((X * ssr_pred).T, expon) * weights_2.T 
    
    biases_deriv = np.dot(ssr_pred.T, expon) * weights_2.T 
    return pd.DataFrame(weights_deriv), np.array(biases_deriv)

In [138]:
derivative_hidden_1(target, output, weights_2, weights, biases, X)

(          0        1
 0  0.208322 -0.09138,
 array([[ 0.20832166, -0.09138002]]))

In [118]:
def derivative_hidden_test(target, output, weights_2, weights, biases, X):
    gradients_w = []
    gradients_b = []
    for i in range(len(weights.T)):
        summa_w = 0.0
        summa_b = 0.0
        for j in range(len(X)):
            summa_w += (-2.) * (target.iloc[j, 0] - output.iloc[j, 0]) * weights_2.iloc[i, 0] * (np.exp(X.iloc[j, 0] * weights.iloc[0, i] + biases[i]) / (1 + np.exp(X.iloc[j, 0] * weights.iloc[0, i] + biases[i]))) * X.iloc[j, 0]
            summa_b += (-2.) * (target.iloc[j, 0] - output.iloc[j, 0]) * weights_2.iloc[i, 0] * (np.exp(X.iloc[j, 0] * weights.iloc[0, i] + biases[i]) / (1 + np.exp(X.iloc[j, 0] * weights.iloc[0, i] + biases[i])))
        gradients_w.append(summa_w)
        gradients_b.append(summa_b)
    return pd.DataFrame(np.array(gradients_w)).T, np.array(gradients_b).T

In [119]:
derivative_hidden_test(target, output, weights_2, weights, biases, X)

(          0        1
 0  0.833287 -0.36552,
 array([ 0.83328666, -0.3655201 ]))

In [232]:
def train(X, target, epochs=10, learning_rate = 0.1):
    
    weights_1, biases_1 = init_weights(X, 2)
    
    hidden1 = neuron_layer(X, weights_1, biases_1, activation=None)
    
    weights_2, biases_2 = init_weights(hidden1, 1, random_state=43)
    output = neuron_layer(hidden1, weights_2, biases_2, activation=None)
    for epoch in range(epochs):
        error = ssr(target, output)
        print(epoch, ": Error:", np.array(error))
        
        ## b3
        gradient_b3 = derivative_of_b3(target, output)
        step_size_b3 = gradient_b3 * learning_rate
        
        biases_2 = np.array(biases_2 - step_size_b3)
        
        ##-- w3, w4  
        gradient_weights = derivative_weights(target, output, hidden1)
        step_size_weights = gradient_weights * learning_rate
        weights_2 = weights_2 - step_size_weights
        
        ##-- w1, w2, b1, b2
        
        ##-- test
        gradient_weights_1_2, gradient_biases_1_2 = derivative_hidden_test(target, output, weights_2, weights_1, biases_1, X)

        ##-- hidden_1
#         gradient_weights_1_2, gradient_biases_1_2 = derivative_hidden_1(target, output, weights_2, weights_1, biases_1, X)
        
        step_size_w_1_2 = gradient_weights_1_2 * learning_rate
        step_size_b_1_2 = gradient_biases_1_2 * learning_rate
        weights_1 = weights_1 - step_size_w_1_2
        biases_1 = biases_1 - step_size_b_1_2
        
        
        hidden1 = neuron_layer(X, weights_1, biases_1, activation=crossenthropy)
        output = neuron_layer(hidden1, weights_2, biases_2, activation=None)
    
    return output

In [233]:
##-- test
train(X, target, epochs=1000, learning_rate=0.1)

0 : Error: [0.16588569]
1 : Error: [0.59321254]
2 : Error: [0.30773356]
3 : Error: [0.2055083]
4 : Error: [0.16598664]
5 : Error: [0.14950436]
6 : Error: [0.14145982]
7 : Error: [0.13643639]
8 : Error: [0.13246939]
9 : Error: [0.12887163]
10 : Error: [0.12541565]
11 : Error: [0.12203316]
12 : Error: [0.11870784]
13 : Error: [0.11543844]
14 : Error: [0.11222691]
15 : Error: [0.10907498]
16 : Error: [0.10598346]
17 : Error: [0.10295239]
18 : Error: [0.0999812]
19 : Error: [0.09706903]
20 : Error: [0.09421482]
21 : Error: [0.09141744]
22 : Error: [0.08867578]
23 : Error: [0.08598881]
24 : Error: [0.08335559]
25 : Error: [0.08077527]
26 : Error: [0.07824717]
27 : Error: [0.07577071]
28 : Error: [0.07334544]
29 : Error: [0.07097104]
30 : Error: [0.06864728]
31 : Error: [0.06637405]
32 : Error: [0.06415129]
33 : Error: [0.06197902]
34 : Error: [0.05985732]
35 : Error: [0.05778626]
36 : Error: [0.05576596]
37 : Error: [0.05379652]
38 : Error: [0.05187802]
39 : Error: [0.05001051]
40 : Error: 

335 : Error: [1.00832702e-05]
336 : Error: [9.84997527e-06]
337 : Error: [9.62213397e-06]
338 : Error: [9.39961689e-06]
339 : Error: [9.18229773e-06]
340 : Error: [8.97005326e-06]
341 : Error: [8.76276322e-06]
342 : Error: [8.56031026e-06]
343 : Error: [8.36257988e-06]
344 : Error: [8.16946034e-06]
345 : Error: [7.9808426e-06]
346 : Error: [7.79662026e-06]
347 : Error: [7.61668947e-06]
348 : Error: [7.4409489e-06]
349 : Error: [7.26929966e-06]
350 : Error: [7.10164522e-06]
351 : Error: [6.93789139e-06]
352 : Error: [6.77794624e-06]
353 : Error: [6.62172004e-06]
354 : Error: [6.46912523e-06]
355 : Error: [6.32007634e-06]
356 : Error: [6.17448994e-06]
357 : Error: [6.03228462e-06]
358 : Error: [5.8933809e-06]
359 : Error: [5.75770121e-06]
360 : Error: [5.62516983e-06]
361 : Error: [5.49571285e-06]
362 : Error: [5.36925813e-06]
363 : Error: [5.24573525e-06]
364 : Error: [5.12507546e-06]
365 : Error: [5.00721166e-06]
366 : Error: [4.89207836e-06]
367 : Error: [4.77961159e-06]
368 : Error: 

639 : Error: [9.11875425e-09]
640 : Error: [8.91200425e-09]
641 : Error: [8.70994333e-09]
642 : Error: [8.51246509e-09]
643 : Error: [8.31946556e-09]
644 : Error: [8.13084312e-09]
645 : Error: [7.94649844e-09]
646 : Error: [7.76633449e-09]
647 : Error: [7.59025639e-09]
648 : Error: [7.41817145e-09]
649 : Error: [7.24998908e-09]
650 : Error: [7.08562073e-09]
651 : Error: [6.92497987e-09]
652 : Error: [6.76798195e-09]
653 : Error: [6.6145443e-09]
654 : Error: [6.46458618e-09]
655 : Error: [6.31802863e-09]
656 : Error: [6.17479451e-09]
657 : Error: [6.03480843e-09]
658 : Error: [5.89799672e-09]
659 : Error: [5.76428735e-09]
660 : Error: [5.63360996e-09]
661 : Error: [5.50589576e-09]
662 : Error: [5.38107755e-09]
663 : Error: [5.25908962e-09]
664 : Error: [5.13986778e-09]
665 : Error: [5.02334929e-09]
666 : Error: [4.90947282e-09]
667 : Error: [4.79817846e-09]
668 : Error: [4.68940762e-09]
669 : Error: [4.58310307e-09]
670 : Error: [4.47920888e-09]
671 : Error: [4.37767037e-09]
672 : Error

941 : Error: [8.98115188e-12]
942 : Error: [8.77764394e-12]
943 : Error: [8.57874743e-12]
944 : Error: [8.38435784e-12]
945 : Error: [8.19437306e-12]
946 : Error: [8.00869326e-12]
947 : Error: [7.82722089e-12]
948 : Error: [7.64986063e-12]
949 : Error: [7.47651927e-12]
950 : Error: [7.30710577e-12]
951 : Error: [7.14153111e-12]
952 : Error: [6.97970831e-12]
953 : Error: [6.82155234e-12]
954 : Error: [6.66698012e-12]
955 : Error: [6.51591045e-12]
956 : Error: [6.36826395e-12]
957 : Error: [6.22396305e-12]
958 : Error: [6.08293195e-12]
959 : Error: [5.94509655e-12]
960 : Error: [5.81038444e-12]
961 : Error: [5.67872484e-12]
962 : Error: [5.55004859e-12]
963 : Error: [5.42428808e-12]
964 : Error: [5.30137725e-12]
965 : Error: [5.18125151e-12]
966 : Error: [5.06384777e-12]
967 : Error: [4.94910433e-12]
968 : Error: [4.83696093e-12]
969 : Error: [4.72735863e-12]
970 : Error: [4.62023987e-12]
971 : Error: [4.51554837e-12]
972 : Error: [4.41322912e-12]
973 : Error: [4.31322837e-12]
974 : Erro

,0
0,0.000001
1,0.999998
2,0.999998
3,0.000001


In [231]:
##-- hidden_1
train(X, target, epochs=1000, learning_rate=0.1)

0 : Error: [0.16588569]
1 : Error: [0.62857346]
2 : Error: [0.36543845]
3 : Error: [0.25043413]
4 : Error: [0.19925095]
5 : Error: [0.17585636]
6 : Error: [0.16456165]
7 : Error: [0.15851201]
8 : Error: [0.15471995]
9 : Error: [0.15189016]
10 : Error: [0.14946493]
11 : Error: [0.14720634]
12 : Error: [0.14501431]
13 : Error: [0.14284775]
14 : Error: [0.14069055]
15 : Error: [0.1385371]
16 : Error: [0.13638606]
17 : Error: [0.13423782]
18 : Error: [0.13209339]
19 : Error: [0.12995399]
20 : Error: [0.12782089]
21 : Error: [0.12569532]
22 : Error: [0.12357848]
23 : Error: [0.12147154]
24 : Error: [0.11937559]
25 : Error: [0.11729172]
26 : Error: [0.11522093]
27 : Error: [0.11316424]
28 : Error: [0.11112257]
29 : Error: [0.10909683]
30 : Error: [0.10708789]
31 : Error: [0.10509657]
32 : Error: [0.10312365]
33 : Error: [0.10116986]
34 : Error: [0.09923589]
35 : Error: [0.09732239]
36 : Error: [0.09542996]
37 : Error: [0.09355917]
38 : Error: [0.09171052]
39 : Error: [0.0898845]
40 : Error: 

355 : Error: [1.4384593e-05]
356 : Error: [1.39605428e-05]
357 : Error: [1.35489701e-05]
358 : Error: [1.31495087e-05]
359 : Error: [1.27618031e-05]
360 : Error: [1.23855084e-05]
361 : Error: [1.20202895e-05]
362 : Error: [1.16658215e-05]
363 : Error: [1.13217885e-05]
364 : Error: [1.09878842e-05]
365 : Error: [1.06638113e-05]
366 : Error: [1.03492808e-05]
367 : Error: [1.00440125e-05]
368 : Error: [9.74773434e-06]
369 : Error: [9.46018217e-06]
370 : Error: [9.18109959e-06]
371 : Error: [8.91023771e-06]
372 : Error: [8.64735493e-06]
373 : Error: [8.39221672e-06]
374 : Error: [8.14459544e-06]
375 : Error: [7.90427008e-06]
376 : Error: [7.67102615e-06]
377 : Error: [7.44465542e-06]
378 : Error: [7.22495577e-06]
379 : Error: [7.011731e-06]
380 : Error: [6.80479068e-06]
381 : Error: [6.60394994e-06]
382 : Error: [6.40902936e-06]
383 : Error: [6.21985475e-06]
384 : Error: [6.03625706e-06]
385 : Error: [5.85807217e-06]
386 : Error: [5.68514081e-06]
387 : Error: [5.51730836e-06]
388 : Error: 

644 : Error: [2.46086462e-09]
645 : Error: [2.38804748e-09]
646 : Error: [2.31738495e-09]
647 : Error: [2.24881328e-09]
648 : Error: [2.18227061e-09]
649 : Error: [2.1176969e-09]
650 : Error: [2.05503389e-09]
651 : Error: [1.99422505e-09]
652 : Error: [1.93521551e-09]
653 : Error: [1.87795204e-09]
654 : Error: [1.82238297e-09]
655 : Error: [1.76845818e-09]
656 : Error: [1.71612899e-09]
657 : Error: [1.66534821e-09]
658 : Error: [1.61607002e-09]
659 : Error: [1.56824996e-09]
660 : Error: [1.52184489e-09]
661 : Error: [1.47681293e-09]
662 : Error: [1.43311345e-09]
663 : Error: [1.39070704e-09]
664 : Error: [1.34955543e-09]
665 : Error: [1.30962149e-09]
666 : Error: [1.27086919e-09]
667 : Error: [1.23326357e-09]
668 : Error: [1.19677069e-09]
669 : Error: [1.16135765e-09]
670 : Error: [1.12699247e-09]
671 : Error: [1.09364416e-09]
672 : Error: [1.06128263e-09]
673 : Error: [1.02987868e-09]
674 : Error: [9.99403972e-10]
675 : Error: [9.69831017e-10]
676 : Error: [9.41133129e-10]
677 : Error

948 : Error: [2.6627123e-13]
949 : Error: [2.58391864e-13]
950 : Error: [2.5074566e-13]
951 : Error: [2.43325718e-13]
952 : Error: [2.36125344e-13]
953 : Error: [2.2913804e-13]
954 : Error: [2.223575e-13]
955 : Error: [2.15777607e-13]
956 : Error: [2.09392422e-13]
957 : Error: [2.03196185e-13]
958 : Error: [1.97183304e-13]
959 : Error: [1.91348353e-13]
960 : Error: [1.85686067e-13]
961 : Error: [1.80191337e-13]
962 : Error: [1.74859203e-13]
963 : Error: [1.69684856e-13]
964 : Error: [1.64663625e-13]
965 : Error: [1.5979098e-13]
966 : Error: [1.55062524e-13]
967 : Error: [1.5047399e-13]
968 : Error: [1.46021238e-13]
969 : Error: [1.41700249e-13]
970 : Error: [1.37507125e-13]
971 : Error: [1.33438082e-13]
972 : Error: [1.29489447e-13]
973 : Error: [1.25657659e-13]
974 : Error: [1.21939259e-13]
975 : Error: [1.18330893e-13]
976 : Error: [1.14829303e-13]
977 : Error: [1.11431331e-13]
978 : Error: [1.08133909e-13]
979 : Error: [1.04934063e-13]
980 : Error: [1.01828906e-13]
981 : Error: [9.8

,0
0,2.348095e-07
1,9.999998e-01
2,9.999998e-01
3,2.348095e-07
